<h2>IS453 Financial Analytics</h2><h2>Lab 4 - Volatility, RAR, and Beta

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

**Documentation for new Pandas functions used**<br>
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.shift.html<br>
https://numpy.org/doc/stable/reference/generated/numpy.reshape.html<br>
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.cov.html


## 1. Calculate annualized volatility

Read 5 years of data for SPY between 2017-2021. Extract the adjusted close prices.



In [2]:
# sample code
tickers = ['SPY']

start_date = '2016-12-30'
end_date = '2022-01-01' 

data1 = yf.download(tickers, start= start_date, end= end_date)

stock_price_5y = data1[['Adj Close']]
stock_price_5y

[*********************100%%**********************]  1 of 1 completed


,Adj Close
Date,
2016-12-30,195.791092
2017-01-03,197.288849
2017-01-04,198.462601
2017-01-05,198.304886
2017-01-06,199.014404
...,...
2021-12-27,456.750946
2021-12-28,456.377655
2021-12-29,456.961517


**Example showing how the pandas df.shift() method works**

In [3]:
# sample code
# the output gets shifted by forward by 1 unit of date period (index)
test_data = stock_price_5y[['Adj Close']]
test_data['shifted']= test_data.shift(1)
test_data

,Adj Close,shifted
Date,,
2016-12-30,195.791092,NaN
2017-01-03,197.288849,195.791092
2017-01-04,198.462601,197.288849
2017-01-05,198.304886,198.462601
2017-01-06,199.014404,198.304886
...,...,...
2021-12-27,456.750946,450.377136
2021-12-28,456.377655,456.750946
2021-12-29,456.961517,456.377655


In [4]:
# to find the ratio of the change in the current price to the previous price
test_data['ratio'] = test_data['Adj Close']/test_data['Adj Close'].shift(1)
test_data

,Adj Close,shifted,ratio
Date,,,
2016-12-30,195.791092,NaN,NaN
2017-01-03,197.288849,195.791092,1.007650
2017-01-04,198.462601,197.288849,1.005949
2017-01-05,198.304886,198.462601,0.999205
2017-01-06,199.014404,198.304886,1.003578
...,...,...,...
2021-12-27,456.750946,450.377136,1.014152
2021-12-28,456.377655,456.750946,0.999183
2021-12-29,456.961517,456.377655,1.001279


**Exercise Q1: Calculate the annualized historical volatility for SPY**

Calculate for periods of the most recent 20 days, 3 months, 1 year (2021) and 5 years (the full data set).<BR>
Note that the calculations must be based on log returns from prices as Rn = log(Cn/Cn-1)

The formula for annualising historical volatility is in the lecture slides.<BR>
  You can assume that there are 252 trading days in a year.

*Hint: For defined periods of data such as x-months or x-years, use .loc to select data ranges to subset the prices. For the 20-day period, it refers to twenty trading days, not calendar days.  So use tail or .iloc instead to select the range.*

 - Which period has the highest annualized historical volatility? <br>
 - Why do you think that period has the highest?</b></font>

**Example: 20-day HV**

In [5]:
# sample code 

# get the most recent 20 days
spy_20d = stock_price_5y['Adj Close'].iloc[-20:]

# take the log of price change(returns) and then apply std()
# default for Pandas std() is sample formula
spy_20d_hv = np.log(spy_20d / spy_20d.shift(1)).std()

# to annualize, multiply by sqrt(252)
ann_spy_20d_hv = spy_20d_hv * np.sqrt(252)
print(f'{ann_spy_20d_hv:.1%}')

16.3%


**Find the HV for other periods**

In [6]:
# your code here
# Note: use the loc method to subset the prices for the period and then apply the formula to find HV

#3 months - Oct to Dec 2021
spy_3m = stock_price_5y['Adj Close'].loc['2021-10-01':'2021-12-31']
spy_3m_hv = np.log(spy_3m / spy_3m.shift(1)).std()
ann_spy_3m_hv = spy_3m_hv * np.sqrt(252)
print(f'3 months is {ann_spy_3m_hv:.1%}')

#1 year - for all 2021
spy_1y = stock_price_5y['Adj Close'].loc['2021-01-01':'2021-12-31']
spy_1y_hv = np.log(spy_1y / spy_1y.shift(1)).std()
ann_spy_1y_hv = spy_1y_hv * np.sqrt(252)
print(f' 1 year is {ann_spy_1y_hv:.1%}')


#5 years
spy_5y = stock_price_5y['Adj Close']
spy_5y_hv = np.log(spy_5y / spy_5y.shift(1)).std()
ann_spy_5y_hv = spy_5y_hv * np.sqrt(252)
print(f' 5 year is {ann_spy_5y_hv:.1%}')


3 months is 13.8%
 1 year is 12.9%
 5 year is 19.0%


- Your answer here

1) 5 years has the highest volatilty 
2) It has a longer timeframe which captures more market events. There is also Covid during the 5 year period which contributed to the high volatility

## 2. Calculate risk adjusted returns

In [7]:
tickers = ['SPY','TLT','EFA']

start_date = '2020-12-31'
end_date = '2022-01-01' 

data2 = yf.download(tickers, start= start_date, end= end_date)

stocks_price1 = data2['Adj Close']
stocks_price1

[*********************100%%**********************]  3 of 3 completed


,EFA,SPY,TLT
Date,,,
2020-12-31,64.694145,353.106567,140.378052
2021-01-04,65.031082,348.299408,140.208984
2021-01-05,65.704987,350.698212,139.167679
2021-01-06,66.458687,352.794922,136.310822
2021-01-07,66.529625,358.036621,135.109344
...,...,...,...
2021-12-27,72.352753,456.750916,134.545486
2021-12-28,72.380180,456.377686,134.012207
2021-12-29,72.352753,456.961487,132.548203


**Exercise Q2: Calculate the Sharpe ratio for each fund**

For the risk-adjusted return, assume that the daily risk-free rate is 0.0001.<br>

- Which fund has the highest RAR
- Which fund has the lowest RAR

In [8]:
# sample code

daily_risk_free_rate = 0.0001

# create the percent returns and delete blank first row
pct_returns1 = stocks_price1.pct_change().dropna()

# find the average daily return values for the instruments
avg_daily_ret = pct_returns1.mean()

# create a new dataframe and add required columns to compute Sharpe ratio
stocks_df_RAR = pd.DataFrame(pct_returns1)

# add the risk free rate column in the df
stocks_df_RAR['RiskFree_Rate'] = daily_risk_free_rate

# compute the average risk free rate value
avg_rf_ret = stocks_df_RAR['RiskFree_Rate'].mean()

# add the excess return columns for each ETF
stocks_df_RAR['Excess_ret_SPY'] = stocks_df_RAR["SPY"] - stocks_df_RAR['RiskFree_Rate']
stocks_df_RAR['Excess_ret_TLT'] = stocks_df_RAR["TLT"] - stocks_df_RAR['RiskFree_Rate']
stocks_df_RAR['Excess_ret_EFA'] = stocks_df_RAR["EFA"] - stocks_df_RAR['RiskFree_Rate']

# the constructed df for RAR calculations
stocks_df_RAR

,EFA,SPY,TLT,RiskFree_Rate,Excess_ret_SPY,Excess_ret_TLT,Excess_ret_EFA
Date,,,,,,,
2021-01-04,0.005208,-0.013614,-0.001204,0.0001,-0.013714,-0.001304,0.005108
2021-01-05,0.010363,0.006887,-0.007427,0.0001,0.006787,-0.007527,0.010263
2021-01-06,0.011471,0.005979,-0.020528,0.0001,0.005879,-0.020628,0.011371
2021-01-07,0.001067,0.014858,-0.008814,0.0001,0.014758,-0.008914,0.000967
2021-01-08,0.009330,0.005698,-0.003228,0.0001,0.005598,-0.003328,0.009230
...,...,...,...,...,...,...,...
2021-12-27,0.007770,0.014152,0.002424,0.0001,0.014052,0.002324,0.007670
2021-12-28,0.000379,-0.000817,-0.003964,0.0001,-0.000917,-0.004064,0.000279
2021-12-29,-0.000379,0.001279,-0.010924,0.0001,0.001179,-0.011024,-0.000479


In [9]:
# sample code

# calculate sharpe ratio for SPY
sharpe_SPY = (avg_daily_ret['SPY'] - avg_rf_ret)/stocks_df_RAR['Excess_ret_SPY'].std()
ann_sharpe_SPY = sharpe_SPY * np.sqrt(252)
print(f'Annualised Sharpe Ratio SPY = {ann_sharpe_SPY:.3f}')


Annualised Sharpe Ratio SPY = 1.816


In [10]:
# your code here

# calculate sharpe ratio for TLT
sharpe_TLT = (avg_daily_ret['TLT'] - avg_rf_ret)/stocks_df_RAR['Excess_ret_TLT'].std()
ann_sharpe_TLT = sharpe_TLT * np.sqrt(252)
print(f'Annualised Sharpe Ratio TLT = {ann_sharpe_TLT:.3f}')

# calculate sharpe ratio for EFA
EFA = (avg_daily_ret['EFA'] - avg_rf_ret)/stocks_df_RAR['Excess_ret_EFA'].std()
ann_EFA = EFA * np.sqrt(252)
print(f'Annualised Sharpe Ratio EFA = {ann_EFA:.3f}')


Annualised Sharpe Ratio TLT = -0.452
Annualised Sharpe Ratio EFA = 0.714


Highest RAR - SPY
Lowest RAR - TLT

## 3. Calculate beta

Beta is a measure of the volatility of an individual asset in comparison to the volatility of the entire market as measured by a benchmark.<BR>
We will use the covariance to calculate.
  
**Exercise Q3: Calculate the 1 year beta for Apple stocks with the SPY market benchmark.**

- Does AAPL have higher or lower risk sensitivity compared to the S&P 500 index (SPY)?

In [11]:
# sample code
# convert the price series to percent returns and drop null values before computing beta

tickers = ['AAPL','SPY']

start_date = '2020-12-31'
end_date = '2022-01-01'

data3 = yf.download(tickers, start= start_date, end= end_date)
stocks_price2 = data3['Adj Close']

stocks_price2_returns = stocks_price2.pct_change().dropna()
stocks_price2_returns

[*********************100%%**********************]  2 of 2 completed


,AAPL,SPY
Date,,
2021-01-04,-0.024720,-0.013614
2021-01-05,0.012364,0.006887
2021-01-06,-0.033661,0.005979
2021-01-07,0.034123,0.014857
2021-01-08,0.008631,0.005697
...,...,...
2021-12-27,0.022975,0.014152
2021-12-28,-0.005767,-0.000817
2021-12-29,0.000502,0.001279


In [12]:
# sample code
# find the covariance, the default for Pandas cov() is sample formula
stocks_price2_returns.cov()

,AAPL,SPY
AAPL,0.000250,0.000089
SPY,0.000089,0.000067


Calculate beta using the formula covar(APPL,SPY)/var(SPY)

*Hint: use .loc to extract the APPL-SPY covariance value for the calculation*

In [13]:
# your code here
appl_spy_covar = stocks_price2_returns.cov().loc['AAPL','SPY']
benchmark_variance = stocks_price2_returns['SPY'].var()
beta = appl_spy_covar / benchmark_variance
beta.round(3)

1.325

- Answer here

higher risk sensitivity

## DIY


### 4. Portfolio construction analysis
You will construct portfolios containing five stocks, selected from the 15 highest weighted S&P 500 stocks.<BR>
Use 5 years worth of data for your analysis - from 1 Jan 2017 through 31 Dec 2021.<BR>
S&P 500 component stocks listed at: https://www.slickcharts.com/sp500

**DIY Q1: Which five stocks would you use to create a “low volatility” portfolio?**

*Hint: For simplicity, only consider the stocks' volatilities. You can ignore their correlations.*<BR>
*Hint: use .sort_values()*<BR>
*Hint: the ticker symbol for Berkshire Hathaway Inc. is "BRK-B"*

In [14]:
# your code here
tickers = ['AAPL', 'MSFT', 'NVDA', 'AMZN', 'META', 'GOOGL', 'AVGO', 'TSLA', 'GOOG', 'BRK-B', 'JPM', 'LLY', 'V', 'UNH', 'XOM'] #the top 15 from the website
start_date = '2017-01-01'
end_date = '2022-01-01'

data = yf.download(tickers, start=start_date, end=end_date)

stock_price_5y = data[['Adj Close']]
stock_price_5y

[*********************100%%**********************]  15 of 15 completed


Adj Close                                                 \
                  AAPL        AMZN       AVGO       BRK-B        GOOG   
Date                                                                    
2017-01-03   26.891960   37.683498  13.966705  163.830002   39.166271   
2017-01-04   26.861862   37.859001  13.867252  164.080002   39.204136   
2017-01-05   26.998463   39.022499  13.648748  163.300003   39.558861   
2017-01-06   27.299458   39.799500  13.829658  163.410004   40.163189   
2017-01-09   27.549496   39.846001  13.859416  162.020004   40.188099   
...                ...         ...        ...         ...         ...   
2021-12-27  177.423660  169.669495  63.024387  296.670013  147.533890   
2021-12-28  176.400436  170.660995  62.530869  298.290009  145.923691   
2021-12-29  176.488968  169.201004  62.868282  299.459991  145.979980   
2021-12-30  175.327988  168.644501  62.164448  299.980011  145.479782   
2021-12-31  174.708176  166.716995  62.195312  299.000000  144.161530   

                                                                        \
                 GOOGL         JPM         LLY        META        MSFT   
Date                                                                     
2017-01-03   40.254574   69.610802   65.001305  116.415298   56.714630   
2017-01-04   40.242619   69.739174   65.105873  118.238335   56.460869   
2017-01-05   40.504169   69.097229   65.863899  120.210800   56.460869   
2017-01-06   41.111465   69.105247   65.933632  122.940376   56.950256   
2017-01-09   41.209614   69.153404   66.456429  124.424713   56.768986   
...                ...         ...         ...         ...         ...   
2021-12-27  147.372253  144.663330  270.209961  344.862640  333.793152   
2021-12-28  146.157181  145.102356  268.288300  344.902466  332.623444   
2021-12-29  146.125275  145.029190  270.219757  341.634979  333.305695   
2021-12-30  145.672424  144.955994  269.084167  343.049561  330.742218   
2021-12-31  144.328796  144.837143  268.084473  335.070068  327.818054   

                                                                      
                 NVDA        TSLA         UNH           V        XOM  
Date                                                                  
2017-01-03   2.512895   14.466000  143.385986   75.214592  62.952065  
2017-01-04   2.571523   15.132667  143.794540   75.829559  62.259445  
2017-01-05   2.506243   15.116667  144.034302   76.718872  61.331326  
2017-01-06   2.539746   15.267333  144.238617   77.778534  61.296719  
2017-01-09   2.642714   15.418667  143.830048   77.343315  60.285496  
...               ...         ...         ...         ...        ...  
2021-12-27  30.893620  364.646667  478.379181  212.668808  55.664379  
2021-12-28  30.271656  362.823334  481.673737  213.079208  55.484497  
2021-12-29  29.951187  362.063324  484.202087  213.196503  54.998821  
2021-12-30  29.536873  356.779999  483.100677  212.903336  54.675030  
2021-12-31  29.362165  352.260010  480.907593  211.769775  55.034794  

[1259 rows x 15 columns]

In [32]:
# calculating the volatility 
stock_price_5y_hv = np.log(stock_price_5y / stock_price_5y.shift(1)).std()
ann_stock_price_5y = stock_price_5y_hv * np.sqrt(252)
print("all the stocks")
print(ann_stock_price_5y.sort_values())
print("===top 5===")
print(ann_stock_price_5y.sort_values()[:5].map('{:.1%}'.format))

all the stocks
SPY      0.189611
BRK-B    0.215351
V        0.266078
GOOG     0.270058
GOOGL    0.270831
MSFT     0.275066
LLY      0.278733
UNH      0.285348
AMZN     0.293605
JPM      0.300160
XOM      0.303926
AAPL     0.304766
META     0.334400
AVGO     0.354182
NVDA     0.472583
TSLA     0.601608
dtype: float64
===top 5===
SPY      19.0%
BRK-B    21.5%
V        26.6%
GOOG     27.0%
GOOGL    27.1%
dtype: object


- Answer here

5 "low volatility stock" - SPY, BRK-B, V, GOOG, GOOGL

**DIY Q2: Which five stocks would you use to create a “best RAR” portfolio?**

Assume a daily risk-free rate of 0.0001.

In [34]:
# your code here
daily_risk_free_rate = 0.0001
pct_ret = stock_price_5y.pct_change().dropna()
avg_daily_ret = pct_ret.mean()
sharpe = (avg_daily_ret - daily_risk_free_rate) / (pct_ret - daily_risk_free_rate).std()
ann_sharpe = sharpe * np.sqrt(252)
print(ann_sharpe.sort_values())
print("===the top 5===")
ann_sharpe.sort_values().tail(5) # the higher the better

XOM     -0.019766
BRK-B    0.550944
JPM      0.554342
META     0.730313
V        0.818104
SPY      0.848017
UNH      0.906211
AVGO     0.959648
GOOGL    0.988210
GOOG     1.009282
LLY      1.056762
AMZN     1.073643
NVDA     1.230854
AAPL     1.301532
TSLA     1.319277
MSFT     1.325049
dtype: float64
===the top 5===


AMZN    1.073643
NVDA    1.230854
AAPL    1.301532
TSLA    1.319277
MSFT    1.325049
dtype: float64

- Answer here

5 "best RAR" - AMZN, NVDA, AAPL, TSLA, MSFT

**DIY Q3: Which five stocks would you use to create a “low market risk sensitivity” portfolio?**

*Hint: make sure SPY is one of your five stocks.*

In [17]:
# your code here
tickers = ['AAPL', 'MSFT', 'NVDA', 'AMZN', 'META', 'GOOGL', 'AVGO', 'TSLA', 'GOOG', 'BRK-B', 'JPM', 'LLY', 'V', 'UNH', 'XOM', 'SPY'] # add SPY inside
start_date = '2017-01-01'
end_date = '2022-01-01'

data = yf.download(tickers, start=start_date, end=end_date)

stock_price_5y = data['Adj Close']
stock_price_5y

[*********************100%%**********************]  16 of 16 completed


,AAPL,AMZN,AVGO,BRK-B,GOOG,GOOGL,JPM,LLY,META,MSFT,NVDA,SPY,TSLA,UNH,V,XOM
Date,,,,,,,,,,,,,,,,
2017-01-03,26.891962,37.683498,13.966709,163.830002,39.166271,40.254574,69.610779,65.001312,116.415298,56.714622,2.512894,197.288818,14.466000,143.385986,75.214592,62.952065
2017-01-04,26.861866,37.859001,13.867246,164.080002,39.204136,40.242619,69.739197,65.105881,118.238335,56.460854,2.571522,198.462555,15.132667,143.794510,75.829559,62.259445
2017-01-05,26.998461,39.022499,13.648749,163.300003,39.558861,40.504169,69.097237,65.863922,120.210800,56.460854,2.506243,198.304871,15.116667,144.034332,76.718903,61.331326
2017-01-06,27.299458,39.799500,13.829658,163.410004,40.163189,41.111469,69.105270,65.933624,122.940376,56.950253,2.539745,199.014359,15.267333,144.238571,77.778526,61.296719
2017-01-09,27.549496,39.846001,13.859420,162.020004,40.188099,41.209614,69.153397,66.456421,124.424713,56.768997,2.642714,198.357468,15.418667,143.830017,77.343323,60.285496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,177.423691,169.669495,63.024376,296.670013,147.533890,147.372253,144.663330,270.209991,344.862640,333.793060,30.893620,456.750946,364.646667,478.379211,212.668793,55.664379
2021-12-28,176.400406,170.660995,62.530861,298.290009,145.923691,146.157166,145.102371,268.288269,344.902466,332.623444,30.271654,456.377686,362.823334,481.673798,213.079254,55.484497
2021-12-29,176.488968,169.201004,62.868279,299.459991,145.979980,146.125275,145.029190,270.219666,341.634979,333.305756,29.951185,456.961517,362.063324,484.202026,213.196503,54.998821


In [ ]:
pct_ret = stock_price_5y.pct_change().dropna()
print(pct_ret.cov())
stock_spy_covar = pct_ret.cov().loc[:,'SPY'] # extract column corresponding to spy
benchmark_variance = pct_ret['SPY'].var()
beta = stock_spy_covar / benchmark_variance
print()
print("===the beta for 5 stocks===")
beta.round(3).sort_values()[:5] # the lower the beta, the less sensitive it is

           AAPL      AMZN      AVGO     BRK-B      GOOG     GOOGL       JPM  \
AAPL   0.000368  0.000225  0.000258  0.000139  0.000215  0.000215  0.000160   
AMZN   0.000225  0.000344  0.000195  0.000089  0.000210  0.000209  0.000091   
AVGO   0.000258  0.000195  0.000488  0.000155  0.000203  0.000206  0.000199   
BRK-B  0.000139  0.000089  0.000155  0.000184  0.000126  0.000125  0.000207   
GOOG   0.000215  0.000210  0.000203  0.000126  0.000289  0.000288  0.000153   
GOOGL  0.000215  0.000209  0.000206  0.000125  0.000288  0.000290  0.000153   
JPM    0.000160  0.000091  0.000199  0.000207  0.000153  0.000153  0.000359   
LLY    0.000114  0.000085  0.000099  0.000092  0.000105  0.000107  0.000093   
META   0.000240  0.000238  0.000222  0.000114  0.000242  0.000243  0.000137   
MSFT   0.000245  0.000226  0.000237  0.000137  0.000229  0.000231  0.000159   
NVDA   0.000341  0.000317  0.000390  0.000168  0.000299  0.000302  0.000195   
SPY    0.000174  0.000135  0.000182  0.000132  0.000

LLY      0.723
BRK-B    0.935
AMZN     0.953
SPY      1.000
UNH      1.039
Name: SPY, dtype: float64

- Answer here

5 "low market risk sensitivity" - LLY, BRK-B, AMZN, SPY, UNH

**DYI Q4:**

It is troublesome to have to try to determine the last business day of the previous year by trial and error. <BR>
There should a way to determine that date using Python functions. Let's use GenAI to help us to learn how.

Use Generative AI to help you write the Python code to return the string of the last weekday for a given year. <BR>
The results of the code for different years should be:<BR>
2022 = '2022-12-30'<BR>
2023 = '2023-12-29'<BR>
2024 = '2024-12-31'

In [19]:
# your successful GenAI prompts here:

# % you are an assistant to a Python software developer
# % ...
# Please write a Python function called get_last_business_day(year) that does the following:

# 1. Takes an integer year as input.
# 2. Determines the date of the last business day (weekday, Monday-Friday) of that year.  This means if December 31st is a Saturday or Sunday, it should return the preceding Friday.
# 3. Returns the date as a string in YYYY-MM-DD format.
# Provide Python code that implements this function, using appropriate Python date and time libraries (like datetime).  Also, include comments in the code to explain each step.
# Here are test cases to verify the function:
# - get_last_business_day(2022) should return 2022-12-30
# - get_last_business_day(2023) should return 2023-12-29
# - get_last_business_day(2024) should return 2024-12-31

In [20]:
# your code here
import datetime

def get_last_business_day(year):
    """
    Returns the date of the last business day (Monday-Friday) of a given year as a string.

    Args:
        year (int): The year for which to find the last business day.

    Returns:
        str: The date of the last business day in 'YYYY-MM-DD' format.
    """
    # 1. Create a datetime.date object for December 31st of the given year.
    last_day_of_year = datetime.date(year, 12, 31)

    # 2. Get the day of the week. Monday is 0 and Sunday is 6.
    day_of_week = last_day_of_year.weekday()

    # 3. Check if December 31st is a Saturday (5) or Sunday (6).
    if day_of_week >= 5: # Saturday or Sunday
        # Calculate how many days to subtract to get to Friday.
        days_to_subtract = day_of_week - 4 # Saturday (5) - 4 = 1 day back to Friday, Sunday (6) - 4 = 2 days back to Friday
        last_business_day = last_day_of_year - datetime.timedelta(days=days_to_subtract)
    else:
        # If it's already a weekday (Monday-Friday), then December 31st is the last business day.
        last_business_day = last_day_of_year

    # 4. Format the datetime.date object into 'YYYY-MM-DD' string.
    return last_business_day.strftime('%Y-%m-%d')

# Test cases to verify the function:
print(f"Last business day of 2022: {get_last_business_day(2022)}") # Expected: 2022-12-30
print(f"Last business day of 2023: {get_last_business_day(2023)}") # Expected: 2023-12-29
print(f"Last business day of 2024: {get_last_business_day(2024)}") # Expected: 2024-12-31

Last business day of 2022: 2022-12-30
Last business day of 2023: 2023-12-29
Last business day of 2024: 2024-12-31
